In [1]:
    
#initial imports
import os
import random
import numpy as np
import pandas as pd
import json
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import keras
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
%matplotlib inline
    
    
# vgg16 transfer learning on the planet dataset
import sys
from numpy import load
from matplotlib import pyplot
from keras import backend
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [2]:
def prepare():
    ######################################################################################################
    #####################################################################################################
    #getting data ready
    #create a dataframe containing the names of all of the jpgs in the full dataset
    fileList = []
    for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy'):
        if file.endswith(".jpg"):
            fileList.append(file)

    df = pd.DataFrame(fileList)

    #also creating an index to use later in extracting the img labels from the json files
    index = df.index

    #formatting the dataframe
    df = df.rename(columns={0: "Id"}) 
    df['labels'] = df.Id
    df['labels'] = df['labels'].str.replace('.jpg','')
    df['tower_wooden'] = ""
    df['tower_lattice'] = ""
    df['tower_tucohy'] = ""
    index = df.index
     ######################################################################################################
    #####################################################################################################
    for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy'):
        if file.endswith(".json"):

            #load file as json object
            filePath = ("C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/"+file)
            f = open(filePath,)
            json_file = json.load(f)
            #get file namme without extension
            file2 = file.replace('.json','')

            #find designated jpeg (row in df)
            #rslt_df = df.loc[df['labels'] == file2]
            apples_indices = index[df['labels'] == file2]

            #parse json file to see if tower_wooden, tower_lattice, or tower_tucohy exists
            #add 0 or 1 to the corresponding dataframe row accordingly

            for element in json_file['shapes']:

                #tower wooden
                bool_wood = any(sd['label']=='tower_wooden' for sd in json_file['shapes'])  
                if bool_wood:
                    df.at[apples_indices,'tower_wooden']= 1
                else:
                    df.at[apples_indices,'tower_wooden']= 0 

                #tower lattice
                bool_lat = any(sd['label']=='tower_lattice' for sd in json_file['shapes']) 
                if bool_lat:
                    df.at[apples_indices,'tower_lattice']= 1
                else:
                    df.at[apples_indices,'tower_lattice']= 0

                #tower tucohy
                #bool_tuc = any(sd['label']=='tower_tucohy' for sd in json_file['shapes']) 
                #if bool_tuc:
                    #df.at[apples_indices,'tower_tucohy']= 0
                #else:
                    #df.at[apples_indices,'tower_tucohy']= 0
   ######################################################################################################
    #####################################################################################################                  df.at[apples_indices,'tower_tucohy']= 0

    #create column containing info for all of the towers
    df['towers'] = df[['tower_wooden', 'tower_lattice']].values.tolist()
    df['tower_labels'] = np.empty((len(df), 0)).tolist()
    for i in range(len(df)):

        if df['towers'][i][0] == 1:
            df['tower_labels'][i].append("tower_wooden")

        if df['towers'][i][1] == 1:
            df['tower_labels'][i].append("tower_lattice")

        #if df['towers'][i][2] == 1:
           # df['tower_labels'][i].append("tower_tucohy")
      ######################################################################################################
    #####################################################################################################        
   #need to split the data into testing, training, and validating
    random.seed(123)
    train, validate, test = \
                  np.split(df.sample(frac=1, random_state=42), 
                           [int(.6*len(df)), int(.8*len(df))])    
    #reading in all of the training images
    train_image = []
    for i in tqdm(range(train.shape[0])):
        img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/'+train.iloc[i][0],target_size=(400,400,3))
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
    X = np.array(train_image)
    #reading in all of the validation images
    valid_image = []
    for i in tqdm(range(validate.shape[0])):
        img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/'+validate.iloc[i][0],target_size=(400,400,3))
        img = image.img_to_array(img)
        img = img/255
        valid_image.append(img)
    X_valid = np.array(valid_image)
    #reading in all of the test images
    test_image = []
    for i in tqdm(range(test.shape[0])):
        img = image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data - Copy/'+test.iloc[i][0],target_size=(400,400,3))
        img = image.img_to_array(img)
        img = img/255
        test_image.append(img)
    X_test = np.array(test_image)
    #need to df['DataFrame Column'] = df['DataFrame Column'].astype(float)first convert the column values from strings to floats
    df['tower_wooden'] = df['tower_wooden'].astype(int)
    df['tower_lattice'] = df['tower_lattice'].astype(int)
    #df['tower_tucohy'] = df['tower_tucohy'].astype(int)
    y = np.array(train.drop(['Id', 'labels','towers','tower_labels','tower_tucohy'],axis=1))
    
    #validation set y
    y_validate = np.array(validate.drop(['Id', 'labels','towers','tower_labels','tower_tucohy'],axis=1))
    #test set y
    y_test = np.array(test.drop(['Id', 'labels','towers','tower_labels','tower_tucohy'],axis=1))
    
    
    X_train=np.asarray(X).astype(np.int)
    y_train=np.asarray(y).astype(np.int)
    X_valid=np.asarray(X_valid).astype(np.int)
    y_valid=np.asarray(y_validate).astype(np.int)
    X_test=np.asarray(X_test).astype(np.int)
    y_test=np.asarray(y_test).astype(np.int)
    
    #special =  image.load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data/'+train.iloc[0][0], target_size=(224, 224))
    return X_train, y_train, X_valid, y_valid, X_test, y_test

In [3]:
X_train, y_train, X_valid, y_valid, X_test, y_test = prepare()

100%|███████████████████████████████████████████████████████████████████████████████| 249/249 [00:01<00:00, 148.21it/s]


In [4]:
# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score
 
# define cnn model
def define_model(in_shape=(400, 400, 3), out_shape=2):
	# load model
	model = VGG16(include_top=False, input_shape=in_shape)
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(out_shape, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Fbeta')
	pyplot.plot(history.history['fbeta'], color='blue', label='train')
	pyplot.plot(history.history['val_fbeta'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness(X_train, y_train, X_valid, y_valid, X_test, y_test):
	# load dataset
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterators
	train_it = datagen.flow(X_train, y_train, batch_size=16)
	valid_it = datagen.flow(X_valid, y_valid, batch_size = 16)
	test_it = datagen.flow(X_test, y_test, batch_size=16)
	# define model
	model = define_model()
	# fit model
	history = model.fit(train_it, steps_per_epoch=len(train_it),
		validation_data=valid_it, validation_steps=len(valid_it), epochs=20, verbose=1)
	# evaluate model
	loss, fbeta = model.evaluate(test_it, steps=len(test_it), verbose=1)
	print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))
	# learning curves
	summarize_diagnostics(history)

In [5]:
run_test_harness(X_train, y_train, X_valid, y_valid, X_test, y_test)

Epoch 1/20
47/47 [==============================] - 910s 19s/step - loss: 6.4204 - fbeta: 0.0537 - val_loss: 0.6356 - val_fbeta: 0.0000e+00
Epoch 2/20
45/47 [===========================>..] - ETA: 29s - loss: 0.6050 - fbeta: 0.0000e+00

KeyboardInterrupt: 